In [1]:
from dbconn import querydb, querydbtopandas 
import pandas as pd 
import datetime
import os

In [2]:
items_encuestas = str(pd.read_excel("items_encuestas.xlsx").item_id.tolist()).strip("[").strip("]")

In [3]:
menor = str(datetime.datetime.today() - datetime.timedelta(days=20))[:7]

mayor = str(datetime.datetime.today())[:7] # - datetime.timedelta(days=20))[:7]

In [4]:
mayor

'2020-05'

In [5]:
menor = "2019-04"

In [11]:
mayor = '2020-05'

In [12]:
data = querydbtopandas(f"""
select  a11.id_item  id,
       a11.id_encuesta  id_encuesta,
       a12.id_afiliado  id_afi_afiliado,
       a12.id_prestador  id_pre_prestador,
       a17.desc_pre_nombre  desc_pre_nombre_efector,
       a18.desc_pre_nombre  desc_pre_nombre,
       a18.id_pre_tipo,   
       a12.id_prest_efector,
       fec_encuesta,
       a11.i_R RESPUESTA
from        DBA.ft_d_prest_encuestas        a11
       join        DBA.ft_prest_encuestas        a12
         on        (a11.id_encuesta = a12.id_encuesta)
       join        DBA.d_pre_prestador        a17
         on        (a12.id_prest_efector = a17.id_pre_prestador)      
       join        DBA.d_pre_prestador        a18
         on        (a12.id_prestador = a18.id_pre_prestador)
where (cast(fec_encuesta as date) >= date("{menor}") and cast(fec_encuesta as date) < date("{mayor}")
and a11.id_item in ({items_encuestas})
and a18.id_pre_tipo in ("E","C","P","I"))
group by        a11.id_item,
       a11.i_R,
       fec_encuesta,
       a11.id_encuesta,
       a12.id_afiliado,
       a12.id_prestador,
       a17.desc_pre_nombre,    
       a18.id_pre_tipo,
       a18.desc_pre_nombre,
       a12.id_prest_efector""")

dict_id = {3: 'Atención Médica',
 2: 'Trato y Cordialidad',
 4: 'Disponibilidad de Turno',
 1: 'Puntualidad',
 5: 'Limpieza y Confort'}   

In [13]:
data.fec_encuesta.unique()

array(['2019-10-03T05:04:00.000000000', '2019-07-09T04:34:00.000000000',
       '2019-07-10T20:17:00.000000000', ...,
       '2019-08-13T07:38:00.000000000', '2019-10-21T18:03:00.000000000',
       '2020-04-07T20:04:00.000000000'], dtype='datetime64[ns]')

In [14]:
data.fec_encuesta = data.fec_encuesta.astype(str).str[:7]

In [15]:
for i in data.fec_encuesta.unique():
    df = data[data["fec_encuesta"] == i]
    df["id"] = df["id"].map(dict_id)

    df_part_equipos = df[df["id_pre_tipo"].isin(["P","E"])][["id","id_encuesta", "id_afi_afiliado", "id_pre_prestador", "desc_pre_nombre", "RESPUESTA" ]]

    df_inst_circ = df[df["id_pre_tipo"].isin(["I","C"])][["id","id_encuesta","id_afi_afiliado", "id_prest_efector", "desc_pre_nombre_efector", "RESPUESTA"]]

    df_inst_circ.rename(columns={"id_prest_efector": "id_pre_prestador", "desc_pre_nombre_efector":"desc_pre_nombre"},inplace=True)

    df = pd.concat([df_part_equipos, df_inst_circ])

    df = df[df["id_pre_prestador"] != 0]

    q_afiliados = df.groupby("id_pre_prestador")["id_afi_afiliado"].nunique().rename_axis('id_pre_prestador').reset_index(name='q_afiliados')

    #tope de evaluación por 5 afiliados
    #prestadores_mas_5_afiliados = q_afiliados[q_afiliados.q_afiliados >= 5].id_pre_prestador.tolist()
    #df = df[df["id_pre_prestador"].isin(prestadores_mas_5_afiliados)]

    q_respuestas = df.groupby("id_pre_prestador")["id_encuesta"].nunique().rename_axis('id_pre_prestador').reset_index(name='q_respuestas')

    df = df.merge(q_respuestas).merge(q_afiliados)

    pivot = pd.pivot_table(df, 
                 index=["id_pre_prestador","desc_pre_nombre", "q_afiliados", "q_respuestas"], 
                 columns="id", 
                 values="RESPUESTA", aggfunc= ["mean", "sum"]).reset_index()
    pivot.columns = ["Prestador", 'Razón social', 'Q afiliados', "Q respuestas", 'Atención Médica m', 'Disponibilidad de Turno m',
               'Limpieza y Confort m', 'Puntualidad m', 'Trato y Cordialidad m',
               'Atención Médica s', 'Disponibilidad de Turno s', 'Limpieza y Confort s',
               'Puntualidad s', 'Trato y Cordialidad s']

    pivot.rename(columns={
                    'Atención Médica s':"Calificación Atención Médica",
                    'Disponibilidad de Turno s':"Calificación Disponibilidad de Turno",
                    'Limpieza y Confort s':"Calificación Limpieza y Confort",
                    'Puntualidad s': "Calificación Puntualidad", 
                    'Trato y Cordialidad s':"Calificación Trato y Cordialidad",
                    'Atención Médica m':"Promedio Calificación Atención Médica", 
                    'Disponibilidad de Turno m':"Promedio Calificación Disponibilidad de Turno", 
                    'Limpieza y Confort m':"Promedio Calificación Limpieza y Confort",
                    'Puntualidad m':"Promedio Calificación Puntualidad",
                    'Trato y Cordialidad m':"Promedio Calificación Trato y Cordialidad"}, inplace=True)

    pivot["Promedio total"] = (pivot["Promedio Calificación Atención Médica"]+
                                  pivot["Promedio Calificación Disponibilidad de Turno"]+
                                  pivot["Promedio Calificación Limpieza y Confort"]+
                                  pivot["Promedio Calificación Puntualidad"]+
                                  pivot["Promedio Calificación Trato y Cordialidad"]) / 5

    pivot= pivot[['Prestador', 'Razón social', 'Q respuestas', 'Q afiliados',
                         'Calificación Atención Médica', 'Promedio Calificación Atención Médica',
                         'Calificación Disponibilidad de Turno','Promedio Calificación Disponibilidad de Turno',
                         'Calificación Limpieza y Confort',   'Promedio Calificación Limpieza y Confort',
                         'Calificación Puntualidad',   'Promedio Calificación Puntualidad',
                         'Calificación Trato y Cordialidad',  'Promedio Calificación Trato y Cordialidad','Promedio total']]
    pivot = pivot.sort_values(by="Promedio total", ascending= True)
    pivot = pivot.round(2)
    
    pivot.to_parquet(f'datos_calificaciones/{i}.gzip',compression='gzip') 
    
    
    #promedios_totales = pd.DataFrame(pivot[['Promedio Calificación Atención Médica', 'Promedio Calificación Disponibilidad de Turno', 'Promedio Calificación Limpieza y Confort', 'Promedio Calificación Puntualidad',   'Promedio Calificación Trato y Cordialidad',  'Promedio total']].mean()).reset_index().rename(columns={0:"Promedios totales", "index": "Calificación"})

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
pivot.Prestador.nunique()

1496

In [52]:
fechas = [i for i in os.listdir('datos_calificaciones/') if len(i) == 12]
completo = pd.DataFrame()
for i in fechas:
    completo = pd.concat([completo, pd.read_parquet(f'datos_calificaciones/{i}')])

completo.Prestador.unique()

In [71]:
pd.Series(completo.Prestador.unique()).to_csv("prestadores_unicos_calificaciones.csv")

# Un mes

In [405]:
her = pd.read_csv("her.csv")

In [407]:
her["Promedio Calificación Atención Médica"]

0          3
1    4,77545
2          3
3    4,77545
4          3
5    4,77545
6          3
7    4,77545
Name: Promedio Calificación Atención Médica, dtype: object

In [410]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=random_y0,
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=time, y=random_y1,
                    mode='lines+markers',
                    name='lines+markers'))
fig.add_trace(go.Scatter(x=time, y=random_y2,
                    mode='markers', name='markers'))

fig.show()

NameError: name 'time' is not defined

In [411]:
import plotly.graph_objects as go

# Add data
month = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
         'August', 'September', 'October', 'November', 'December']
high_2000 = [32.5, 37.6, 49.9, 53.0, 69.1, 75.4, 76.5, 76.6, 70.7, 60.6, 45.1, 29.3]
low_2000 = [13.8, 22.3, 32.5, 37.2, 49.9, 56.1, 57.7, 58.3, 51.2, 42.8, 31.6, 15.9]
high_2007 = [36.5, 26.6, 43.6, 52.3, 71.5, 81.4, 80.5, 82.2, 76.0, 67.3, 46.1, 35.0]
low_2007 = [23.6, 14.0, 27.0, 36.8, 47.6, 57.7, 58.9, 61.2, 53.3, 48.5, 31.0, 23.6]
high_2014 = [28.8, 28.5, 37.0, 56.8, 69.7, 79.7, 78.5, 77.8, 74.1, 62.6, 45.3, 39.9]
low_2014 = [12.7, 14.3, 18.6, 35.5, 49.9, 58.0, 60.0, 58.6, 51.7, 45.2, 32.2, 29.1]

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=month, y=high_2014, name='High 2014',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x=month, y=low_2014, name = 'Low 2014',
                         line=dict(color='royalblue', width=4)))
fig.add_trace(go.Scatter(x=month, y=high_2007, name='High 2007',
                         line=dict(color='firebrick', width=4,
                              dash='dash') # dash options include 'dash', 'dot', and 'dashdot'
))
fig.add_trace(go.Scatter(x=month, y=low_2007, name='Low 2007',
                         line = dict(color='royalblue', width=4, dash='dash')))
fig.add_trace(go.Scatter(x=month, y=high_2000, name='High 2000',
                         line = dict(color='firebrick', width=4, dash='dot')))
fig.add_trace(go.Scatter(x=month, y=low_2000, name='Low 2000',
                         line=dict(color='royalblue', width=4, dash='dot')))

# Edit the layout
fig.update_layout(title='Average High and Low Temperatures in New York',
                   xaxis_title='Month',
                   yaxis_title='Temperature (degrees F)')


fig.show()

In [ ]:
fig.write_html("califica.html")